# ResNet

In [1]:
import sys
sys.path.append('..')

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [2]:
def conv3x3(in_channel, out_channel, stride=1):
    return nn.Conv2d(in_channel, out_channel, 3, stride=stride, padding=1, bias=False)

In [3]:
class residual_block(nn.Module): #/ri'zidrul/
    def __init__(self, in_channel, out_channel, same_shape=True):
        super(residual_block, self).__init__()
        self.same_shape = same_shape
        stride=1 if self.same_shape else 2 #控制stride
        
        self.conv1 = conv3x3(in_channel, out_channel, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channel)
        
        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel, 1, stride=stride)
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)
        
        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(x+out, True) #residual的核心,nn.ReLU(out+x)

In [5]:
# 输入输出形状相同
test_net = residual_block(32, 32)
test_x = Variable(torch.zeros(1, 32, 96, 96))
print('input: {}'.format(test_x.shape))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

input: torch.Size([1, 32, 96, 96])
output: torch.Size([1, 32, 96, 96])


In [6]:
# 输入输出形状不同
test_net = residual_block(3, 32, False)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print('input: {}'.format(test_x.shape))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

input: torch.Size([1, 3, 96, 96])
output: torch.Size([1, 32, 48, 48])


In [7]:
class resnet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(resnet, self).__init__()
        self.verbose = verbose
        
        self.block1 = nn.Conv2d(in_channel, 64, 7, 2)
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(3, 2),
            residual_block(64, 64),
            residual_block(64, 64) )
        
        self.block3 = nn.Sequential(residual_block(64, 128, False),
            residual_block(128, 128))
        
        self.block4 = nn.Sequential(residual_block(128, 256, False),
            residual_block(256, 256) )
        
        self.block5 = nn.Sequential( residual_block(256, 512, False),
                                    residual_block(512, 512),
                                    nn.AvgPool2d(3) )
        
        self.classifier = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [8]:
test_net = resnet(3, 10, True)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

block 1 output: torch.Size([1, 64, 45, 45])
block 2 output: torch.Size([1, 64, 22, 22])
block 3 output: torch.Size([1, 128, 11, 11])
block 4 output: torch.Size([1, 256, 6, 6])
block 5 output: torch.Size([1, 512, 1, 1])
output: torch.Size([1, 10])


In [9]:
from utils import train

def data_tf(x):
    x = x.resize((96, 96), 2) # 将图片放大到 96 x 96
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1)) # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x
     
train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = resnet(3, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


train(net, train_data, test_data, 20, optimizer, criterion)

Epoch 0. Train Loss: 1.434532, Train Acc: 0.475523, Valid Loss: 2.177341, Valid Acc: 0.336729, Time 00:01:38
Epoch 1. Train Loss: 0.987843, Train Acc: 0.650675, Valid Loss: 1.663084, Valid Acc: 0.464102, Time 00:01:45
Epoch 2. Train Loss: 0.765679, Train Acc: 0.732836, Valid Loss: 0.971504, Valid Acc: 0.657634, Time 00:01:44
Epoch 3. Train Loss: 0.605976, Train Acc: 0.788263, Valid Loss: 1.015542, Valid Acc: 0.662975, Time 00:01:44
Epoch 4. Train Loss: 0.477899, Train Acc: 0.832820, Valid Loss: 1.520569, Valid Acc: 0.580498, Time 00:01:44
Epoch 5. Train Loss: 0.369632, Train Acc: 0.874261, Valid Loss: 1.213158, Valid Acc: 0.645075, Time 00:01:44
Epoch 6. Train Loss: 0.277539, Train Acc: 0.906050, Valid Loss: 0.870092, Valid Acc: 0.727057, Time 00:01:45
Epoch 7. Train Loss: 0.197819, Train Acc: 0.931546, Valid Loss: 1.553567, Valid Acc: 0.605716, Time 00:01:45
Epoch 8. Train Loss: 0.152319, Train Acc: 0.948909, Valid Loss: 1.119592, Valid Acc: 0.693137, Time 00:04:36
Epoch 9. Train Loss